<a href="https://colab.research.google.com/github/otanet/NLP_seminar_20201022/blob/main/t5_summarization_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T5を用いた要約の例
*   本ノートブックでは，要約データにfinetuneされたT5モデルで要約を生成する例を示します．
*   huggingfaceの[transformers](https://github.com/huggingface/transformers)を使用します．
*   T5をfinetuneする例は含まれません（T5は事前学習の時点で要約データも学習しています）
*   T5を独自データでfinetuneする例については[こちら](https://github.com/huggingface/transformers/tree/master/examples/seq2seq)を参考にしてください




## 環境のセットアップ

In [ ]:
!pip install transformers==3.3.1

     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 3.0MB 7.6MB/s 
     |████████████████████████████████| 1.1MB 42.8MB/s 
     |████████████████████████████████| 890kB 43.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=72a47d9a3efdcbc6095da3ba845d8a8772c8f0138a64c1b0c77145dd129c1ab9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

## モデル・トークナイザ読み込み

In [ ]:
model = T5ForConditionalGeneration.from_pretrained('t5-small', return_dict=True)
tokenizer = T5Tokenizer.from_pretrained('t5-small')

## ソーステキストの指定

*   ソーステキストを指定して，トークナイズ＆id化する
*   下記のARTICLE_TO_SUMMARIZE =""の中身を自由に変更して要約を実行
*   要約タスクの場合は，先頭に"summarize: "をつける
*   今回は例文として[こちら](https://edition.cnn.com/2020/10/19/politics/donald-trump-joe-biden-election-2020-coronavirus-fauci-masks/index.html)のニューステキスト（一部）を用いる




In [ ]:
ARTICLE_TO_SUMMARIZE = "summarize: " + "(CNN)President Donald Trump and the pandemic he is supposed to be fighting are running out of control with the two weeks until Election Day shaping up as among the most ugly and divisive periods ever ahead of a presidential vote. He's on a fresh collision course with Dr. Anthony Fauci, who's publicly questioning why Trump thinks mask wearing is weak after a wild weekend that saw the President, who's trailing former Vice President Joe Biden in the polls and still playing to his base, pack swing state rallies that flouted his government's Covid-19 protocols."
input_ids = tokenizer.encode(ARTICLE_TO_SUMMARIZE, return_tensors="pt", max_length=1024, truncation=True) 
print(input_ids)

tensor([[21603,    10,    41,   254, 17235,    61,   345, 15704,  7459,  2523,
            11,     8,  2131,   221,  3113,     3,    88,    19,  5741,    12,
            36,  6237,    33,  1180,    91,    13,   610,    28,     8,   192,
          1274,   552, 19488,  1430, 20266,    95,    38,   859,     8,   167,
         18754,    11,  1227,  3466,   757,  8811,   664,  2177,    13,     3,
             9, 13074,  2902,     5,   216,    31,     7,    30,     3,     9,
          1434, 16345,   503,    28,   707,     5, 11016,  1699,  6809,     6,
           113,    31,     7, 11652,   822,    53,   572,  2523,   317,     7,
          8181,  5119,    19,  5676,   227,     3,     9,  3645,  1851,    24,
          1509,     8,  1661,     6,   113,    31,     7,  5032,    53,  1798,
          8236,  1661,  4967,  2106,   537,    16,     8,  5492,     7,    11,
           341,  1556,    12,   112,  1247,     6,  4153,  7180,   538, 26930,
             7,    24,  8882,  2810,    26,   112,  

## 要約の生成

*   num_beamsを変えることで，どの範囲まで探索するかを変えることができる（この値を大きくするほど推論は遅くなる）
*   max_lengthで要約の出力最大長を指定する．この長さを超える場合は強制的に終了する

In [ ]:
summary_ids = model.generate(input_ids, num_beams=4, max_length=120, early_stopping=True)
decoded_summary =[tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
print(decoded_summary)

['the two weeks until election day shaping up as one of the most ugly and divisive periods ever ahead of a presidential vote . the president is trailing former vice president Joe Biden in the polls and still playing to his base .']
